In [1]:
# Install necessary libraries
!pip install requests pandas

In [2]:
# Import libraries
import requests
import pandas as pd
import json

In [3]:
# Set the base URL for the GWAS Catalog REST API
BASE_URL = "https://wwwdev.ebi.ac.uk/gwas/beta/rest/api"

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

print("Setup complete. Libraries are installed and imported.")

Setup complete. Libraries are installed and imported.


In [4]:
def gwas_api_request(endpoint, params=None):
    """
    Performs a GET request to the GWAS Catalog REST API.

    Args:
        endpoint (str): The API endpoint to call (e.g., '/v2/studies').
        params (dict, optional): A dictionary of query parameters. Defaults to None.

    Returns:
        dict: The JSON response from the API as a Python dictionary, or None if the request fails.
    """
    full_url = f"{BASE_URL}{endpoint}"
    
    try:
        # Make the GET request
        response = requests.get(full_url, params=params)
        
        # Check if the request was successful (HTTP status code 200)
        if response.status_code == 200:
            return response.json()
        else:
            # Print an error message if the request was not successful
            print(f"Error: Received status code {response.status_code}")
            print(f"URL: {response.url}")
            print(f"Response: {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        # Handle network-related errors
        print(f"An error occurred: {e}")
        return None

print("Helper function 'gwas_api_request' is defined.")

Helper function 'gwas_api_request' is defined.


In [5]:
import time

def get_all_variants_for_trait(trait_name):
    """
    Fetches all unique variant rsIDs for a given trait by handling API pagination.

    Args:
        trait_name (str): The name of the trait to query.

    Returns:
        set: A set of unique rsID strings associated with the trait.
    """
    variants = set()
    current_page = 0
    total_pages = 1
    
    print(f"--- Starting search for '{trait_name}' ---")

    # Loop through all pages of the API results
    while current_page < total_pages:
        endpoint = "/v2/associations"
        params = {
            "efo_trait": trait_name, 
            "size": 200, 
            "page": current_page
        }
        
        # Make the API call
        data = gwas_api_request(endpoint, params)
        
        if data and '_embedded' in data:
            associations_list = data['_embedded']['associations']
            
            # On the first request, find out the total number of pages
            if current_page == 0:
                total_pages = data['page']['totalPages']
                print(f"Found {data['page']['totalElements']} total associations across {total_pages} pages.")

            # Extract the rsID from each association
            for association in associations_list:
                if 'snp_risk_allele' in association and association['snp_risk_allele']:
                    # e.g., from ['rs123-A'], get 'rs123'
                    risk_allele_str = association['snp_risk_allele'][0]
                    rs_id = risk_allele_str.split('-')[0]
                    if rs_id.startswith('rs'):
                        variants.add(rs_id)
            
            print(f"Page {current_page + 1}/{total_pages} processed. Found {len(variants)} unique variants so far.")
            current_page += 1
            time.sleep(0.1) # Be polite to the API by adding a small delay
        else:
            print("No more data or an error occurred. Stopping.")
            break
            
    print(f"--- Finished fetching for '{trait_name}'. Found {len(variants)} total unique variants. ---\n")
    return variants

print("Helper function 'get_all_variants_for_trait' is defined.")

Helper function 'get_all_variants_for_trait' is defined.


In [6]:
def get_all_associations_for_trait(trait_name):
    """
    Fetches all association objects for a given trait by handling API pagination.

    Args:
        trait_name (str): The name of the trait to query.

    Returns:
        list: A list of all association dictionaries for the trait, sorted by p-value.
    """
    all_associations = []
    current_page = 0
    total_pages = 1  # Initialize to 1 to start the loop
    
    print(f"--- Starting search for all associations related to '{trait_name}' ---")

    # Loop through all pages of the API results
    while current_page < total_pages:
        endpoint = "/v2/associations"
        params = {
            "efo_trait": trait_name, 
            "sort": "p_value",  # Sort by significance from the start
            "direction": "asc",
            "size": 200, 
            "page": current_page
        }
        
        data = gwas_api_request(endpoint, params)
        
        if data and '_embedded' in data:
            associations_list = data['_embedded']['associations']
            all_associations.extend(associations_list)
            
            if current_page == 0:
                total_pages = data['page']['totalPages']
                print(f"Found {data['page']['totalElements']} total associations across {total_pages} pages.")

            print(f"Page {current_page + 1}/{total_pages} processed. Collected {len(all_associations)} associations so far.")
            current_page += 1
            time.sleep(0.1) # Be polite to the API
        else:
            print("No more data or an error occurred. Stopping.")
            break
            
    print(f"--- Finished fetching. Found {len(all_associations)} total associations. ---\n")
    return all_associations

print("Helper function 'get_all_associations_for_trait' is defined.")

Helper function 'get_all_associations_for_trait' is defined.


In [7]:
def get_all_genes_for_trait (trait):    
    # Use a set to store unique gene names automatically
    all_genes = set()
    current_page = 0
    total_pages = 1 # Initialize to 1 to start the loop
    
    # Loop through all pages of the API results
    while current_page < total_pages:
        # Define the endpoint and parameters
        endpoint = "/v2/associations"
        params = {
            "efo_trait": disease_of_interest,
            "size": 200,  # Request a larger size per page for efficiency
            "page": current_page
        }
    
        # Make the API call
        associations_data = gwas_api_request(endpoint, params)
    
        # Check if the request was successful and contains data
        if associations_data and '_embedded' in associations_data:
            # Extract the list of associations
            associations_list = associations_data['_embedded']['associations']
            
            # Update the total number of pages from the first response
            if current_page == 0:
                total_pages = associations_data['page']['totalPages']
                print(f"Found {associations_data['page']['totalElements']} total associations across {total_pages} pages.")
    
            # Process each association in the current page
            for association in associations_list:
                # Check if 'mapped_genes' exists and is not empty
                if 'mapped_genes' in association and association['mapped_genes']:
                    # The field is a list, e.g., ['GENE1', 'GENE2,GENE3']
                    for gene_string in association['mapped_genes']:
                        # A single string can have multiple comma-separated genes
                        genes = gene_string.split(',')
                        for gene in genes:
                            if gene.strip():  # Ensure the gene name isn't empty
                                all_genes.add(gene.strip())
    
            print(f"Processed page {current_page + 1} of {total_pages}. Found {len(all_genes)} unique genes so far.")
            current_page += 1
        else:
            # Stop if there's no more data or an error occurs
            print("No more data found or an error occurred. Stopping.")
            break

        return all_genes

In [8]:
def get_all_associations_for_accession_id(accession_id):
    """
    Fetches all association objects for a given trait by handling API pagination.

    Args:
        accession_id (str): The id of the study

    Returns:
        list: A list of all association dictionaries for the study, sorted by p-value.
    """
    all_associations = []
    current_page = 0
    total_pages = 1  # Initialize to 1 to start the loop
    
    print(f"--- Starting search for all associations related to '{accession_id}' ---")

    # Loop through all pages of the API results
    while current_page < total_pages:
        endpoint = "/v2/associations"
        params = {
            "accession_id": accession_id, 
            "sort": "p_value",  # Sort by significance from the start
            "direction": "asc",
            "size": 200, 
            "page": current_page
        }
        
        data = gwas_api_request(endpoint, params)
        
        if data and '_embedded' in data:
            associations_list = data['_embedded']['associations']
            all_associations.extend(associations_list)
            
            if current_page == 0:
                total_pages = data['page']['totalPages']
                print(f"Found {data['page']['totalElements']} total associations across {total_pages} pages.")

            print(f"Page {current_page + 1}/{total_pages} processed. Collected {len(all_associations)} associations so far.")
            current_page += 1
            time.sleep(0.1) # Be polite to the API
        else:
            print("No more data or an error occurred. Stopping.")
            break
            
    print(f"--- Finished fetching. Found {len(all_associations)} total associations. ---\n")
    return all_associations

print("Helper function 'get_all_associations_for_accession_id' is defined.")

Helper function 'get_all_associations_for_accession_id' is defined.


### Question 1: What variants are associated with a specific disease, e.g. type 2 diabetes?

In [9]:
# The disease we are interested in
disease_of_interest = "type 2 diabetes mellitus"

# Define the endpoint and parameters for the query
endpoint = "/v2/associations"
params = {
    "efo_trait": disease_of_interest,
    "size": 10  # Let's get the top 10 results for this example
}

# Make the API call using our helper function
associations_data = gwas_api_request(endpoint, params)

# Process and display the response
if associations_data and '_embedded' in associations_data:
    # Extract the list of associations from the '_embedded' key
    associations_list = associations_data['_embedded']['associations']
    
    # Convert the list into a pandas DataFrame
    associations_df = pd.DataFrame(associations_list)
    
    print(f"Found {associations_data['page']['totalElements']} variants associated with '{disease_of_interest}'.")
    print("Displaying the first 10 results:")
    
    # Parse the 'snp_risk_allele' field. It's a list containing a string like 'rsID-Allele'
    # Extract the string 'rsID-Allele' from the list
    risk_allele_str = associations_df['snp_risk_allele'].str[0]
    
    # Split the string into the variant ID and the allele base
    split_allele = risk_allele_str.str.split('-', n=1, expand=True)
    associations_df['variant_rsID'] = split_allele[0]
    associations_df['risk_allele_base'] = split_allele[1]
    
    # Select, rename, and display the most relevant columns for clarity
    display_df = associations_df[[
        'variant_rsID', 
        'risk_allele_base',
        'p_value', 
        'risk_frequency', 
        'accession_id',
        'mapped_genes'
    ]]

    display(display_df)
else:
    print(f"No associations found for '{disease_of_interest}' or an error occurred.")


Found 7651 variants associated with 'type 2 diabetes mellitus'.
Displaying the first 10 results:


,variant_rsID,risk_allele_base,p_value,risk_frequency,accession_id,mapped_genes
0,rs16989540,C,1.000000e-11,0.874914,GCST90528074,[DEPDC5]
1,rs58223766,C,3.000000e-09,0.796662,GCST90528074,"[SLC16A13, SLC16A11]"
2,rs1421085,T,1.000000e-13,0.750703,GCST90528074,[FTO]
3,rs10830963,C,5.000000e-11,0.78925,GCST90528074,[MTNR1B]
4,rs79878066,C,2.000000e-41,0.792611,GCST90528074,[KCNQ1]
5,rs34872471,T,3.000000e-46,0.739357,GCST90528074,[TCF7L2]
6,rs10882099,T,8.000000e-11,0.645401,GCST90528074,"[HHEX, Y_RNA]"
7,rs12780155,T,7.000000e-11,0.758083,GCST90528074,[CDC123]
8,rs12344703,T,2.000000e-08,0.787072,GCST90528074,[MOB3B]
9,rs12555274,G,7.000000e-10,0.711345,GCST90528074,[CDKN2B-AS1]


### Question 2: What studies are available for "type 2 diabetes" in the "UKB" cohort?

To answer this, we'll use the `/v2/studies` endpoint. This endpoint can be filtered by multiple criteria simultaneously. We will provide both the `disease_trait` ("type 2 diabetes") and the `cohort` ("UKB") as parameters to narrow down the results.

In [10]:
# The trait and cohort we are interested in
disease_of_interest = "type 2 diabetes"
cohort_of_interest = "UKB" # UK Biobank

# Define the endpoint and parameters for the query
endpoint = "/v2/studies"
params = {
    "disease_trait": disease_of_interest,
    "cohort": cohort_of_interest,
    "size": 20 # Get up to 20 results
}

# Make the API call using our helper function
studies_data = gwas_api_request(endpoint, params)

# Process and display the response
if studies_data and '_embedded' in studies_data:
    # Extract the list of studies from the '_embedded' key
    studies_list = studies_data['_embedded']['studies']
    
    # Convert the list of studies into a pandas DataFrame for nice display
    studies_df = pd.DataFrame(studies_list)
    
    print(f"Found {studies_data['page']['totalElements']} studies for '{disease_of_interest}' in the '{cohort_of_interest}' cohort.")
    print("Displaying results:")
    
    # Display the DataFrame with selected, useful columns
    display(studies_df[[
        'accession_id', 
        'pubmed_id', 
        'disease_trait', 
        'initial_sample_size', 
        'cohort'
    ]])
else:
    print(f"No studies found for '{disease_of_interest}' in the '{cohort_of_interest}' cohort or an error occurred.")

Found 8 studies for 'type 2 diabetes' in the 'UKB' cohort.
Displaying results:


,accession_id,pubmed_id,disease_trait,initial_sample_size,cohort
0,GCST90444202,39379762,Type 2 diabetes,"51,256 African, African American, East Asian, ...","[UKB, MGBB, GERA, AllofUs]"
1,GCST90302887,37377600,Type 2 diabetes,"22,670 British ancestry cases, 313,404 British...",[UKB]
2,GCST90077724,34662886,Type 2 diabetes,"3,497 European ancestry cases, 328,257 Europea...",[UKB]
3,GCST90132186,35551307,Type 2 diabetes,"40,737 South Asian ancestry individuals","[Other, ITH, LOLIPOP, PROMIS, RHS, SINDI, UKB]"
4,GCST90132184,35551307,Type 2 diabetes,"251,740 European ancestry individuals","[BIOME, DECODE, DGDG, DGI, EGCUT, EPIC, FHS, F..."
5,GCST90100587,34862199,Type 2 diabetes,"33,139 European ancestry cases, 279,507 Europe...","[UKB, FUSION, WTCCC, GERA, MGBB, other]"
6,GCST90018926,34594039,Type 2 diabetes,"38,841 European ancestry cases, 451,248 Europe...","[BBJ, UKB, FinnGen]"
7,GCST90038634,33959723,Type 2 diabetes,"3,260 cases, 481,338 controls",[UKB]


### Question 3: What are the most significant associations for a specific SNP? eg: significant associations for rs1050316

To answer this, we will again use the `/v2/associations` endpoint. This time, we'll filter using the `rs_id` parameter to specify our variant of interest.

To find the **most significant** results, we will instruct the API to sort the data by `p_value` in `asc` (ascending) order. This puts the associations with the smallest p-values (highest significance) at the top of the list.

In [11]:
# The SNP we are interested in
snp_of_interest = "rs1050316"

# Define the endpoint and parameters for the query
endpoint = "/v2/associations"
params = {
    "rs_id": snp_of_interest,
    "sort": "p_value",  # Sort by p-value
    "direction": "asc", # Sort in ascending order (most significant first)
    "size": 10          # Get the top 10 most significant results
}

# Make the API call
associations_data = gwas_api_request(endpoint, params)

# Process and display the response
if associations_data and '_embedded' in associations_data:
    # Extract the list of associations
    associations_list = associations_data['_embedded']['associations']
    
    # Convert to a pandas DataFrame
    associations_df = pd.DataFrame(associations_list)
    
    print(f"Found {associations_data['page']['totalElements']} associations for SNP '{snp_of_interest}'.")
    print("Displaying the 10 most significant results:")
    
    # The 'efo_traits' column is a list of dictionaries.
    # This helper function extracts the trait name for cleaner display.
    def extract_trait(traits_list):
        if traits_list and isinstance(traits_list, list) and len(traits_list) > 0:
            return traits_list[0].get('efo_trait', 'N/A')
        return 'N/A'

    associations_df['trait_name'] = associations_df['efo_traits'].apply(extract_trait)

    # Display the DataFrame with selected and renamed columns
    display(associations_df[[
        'p_value',
        'trait_name',
        'risk_frequency',
        'accession_id',
        'first_author'
    ]])
else:
    print(f"No associations found for SNP '{snp_of_interest}' or an error occurred.")

Found 12 associations for SNP 'rs1050316'.
Displaying the 10 most significant results:


,p_value,trait_name,risk_frequency,accession_id,first_author
0,9.000000e-30,TPE interval measurement,0.347,GCST010346,Ramirez J
1,6.000000e-29,pain,NR,GCST90104572,Mocci E
2,4.000000e-22,pain,NR,GCST90104573,Mocci E
3,3.000000e-21,TPE interval measurement,0.346,GCST010346,Ramirez J
4,2.000000e-15,blood protein amount,0.3621,GCST90090292,Gudjonsson A
5,2.000000e-14,platelet crit,0.6515,GCST004607,Astle WJ
6,3.000000e-12,TPE interval measurement,0.346,GCST010346,Ramirez J
7,2.000000e-11,Headache,NR,GCST005337,Meng W
8,3.000000e-11,body height,NR,GCST90435412,Shi S
9,4.000000e-11,platelet count,0.6515,GCST004603,Astle WJ


### Question 4: Which genes are associated with type 2 diabetes?

To answer this, there isn't a direct endpoint to look up genes by disease. Instead, we can find the answer by:
1. Fetching all the associations for a given disease (e.g., "type 2 diabetes").
2. Extracting the mapped_genes from each of those associations.
3. Because there can be thousands of associations for a common disease, we will need to loop through multiple pages of API results to build a comprehensive list.

In [12]:
# The disease we want to find associated genes for
disease_of_interest = "type 2 diabetes mellitus"
print(f"Searching for genes associated with '{disease_of_interest}'...")

# # Use a set to store unique gene names automatically
# all_genes = set()
# current_page = 0
# total_pages = 1 # Initialize to 1 to start the loop

# # Loop through all pages of the API results
# while current_page < total_pages:
#     # Define the endpoint and parameters
#     endpoint = "/v2/associations"
#     params = {
#         "efo_trait": disease_of_interest,
#         "size": 200,  # Request a larger size per page for efficiency
#         "page": current_page
#     }

#     # Make the API call
#     associations_data = gwas_api_request(endpoint, params)

#     # Check if the request was successful and contains data
#     if associations_data and '_embedded' in associations_data:
#         # Extract the list of associations
#         associations_list = associations_data['_embedded']['associations']
        
#         # Update the total number of pages from the first response
#         if current_page == 0:
#             total_pages = associations_data['page']['totalPages']
#             print(f"Found {associations_data['page']['totalElements']} total associations across {total_pages} pages.")

#         # Process each association in the current page
#         for association in associations_list:
#             # Check if 'mapped_genes' exists and is not empty
#             if 'mapped_genes' in association and association['mapped_genes']:
#                 # The field is a list, e.g., ['GENE1', 'GENE2,GENE3']
#                 for gene_string in association['mapped_genes']:
#                     # A single string can have multiple comma-separated genes
#                     genes = gene_string.split(',')
#                     for gene in genes:
#                         if gene.strip():  # Ensure the gene name isn't empty
#                             all_genes.add(gene.strip())

#         print(f"Processed page {current_page + 1} of {total_pages}. Found {len(all_genes)} unique genes so far.")
#         current_page += 1
#     else:
#         # Stop if there's no more data or an error occurs
#         print("No more data found or an error occurred. Stopping.")
#         break

all_genes = get_all_genes_for_trait(disease_of_interest)

# --- Display the final results ---
print("\n--- Search Complete ---")
if all_genes:
    # Convert the set to a sorted list for clean display
    sorted_genes = sorted(list(all_genes))
    print(f"Found a total of {len(sorted_genes)} unique genes associated with '{disease_of_interest}'.")
    
    # Displaying the first 100 genes as an example
    print("Example genes:", sorted_genes[:100])
else:
    print(f"Could not find any genes associated with '{disease_of_interest}'.")

Searching for genes associated with 'type 2 diabetes mellitus'...
Found 7651 total associations across 39 pages.
Processed page 1 of 39. Found 226 unique genes so far.

--- Search Complete ---
Found a total of 226 unique genes associated with 'type 2 diabetes mellitus'.
Example genes: ['ABHD17A', 'ACBD4', 'ACE', 'ADA', 'ADARB1', 'ADCY5', 'AIDAP3', 'ANGPTL4', 'APOE', 'ARSG', 'ARVCF', 'ASCC2', 'ASXL1', 'ATP1B2', 'ATP2A3', 'ATP5F1E', 'ATXN10', 'BACE2', 'BBLNP1', 'BCL2', 'BPTF', 'CBARP', 'CBARP-DT', 'CBFA2T2', 'CBX1', 'CDC123', 'CDH7', 'CDK12', 'CDKAL1', 'CDKN2B-AS1', 'CEBPB', 'CHMP4B', 'CHRNE', 'CLEC10A', 'CRTC1', 'CYB5R3', 'CYCSP55', 'CYTH1', 'DCAF7', 'DCC', 'DDX52', 'DEPDC5', 'DPEP1', 'DTD1', 'EMILIN2', 'EP300', 'EP300-AS1', 'ERN1', 'ETS2', 'EXOC3L2', 'EYA2', 'FARSA', 'FBXO46', 'FCGRT', 'FN3KRP', 'FTO', 'GALK1', 'GIP', 'GIPR', 'GLIS3', 'GLP2R', 'GNAS', 'GNAS-AS1', 'GRP', 'HHEX', 'HMGA1', 'HNF1B', 'HNF4A', 'HNF4A-AS1', 'HSD17B1', 'HSD17B1-AS1', 'IGF2BP1', 'IGF2BP2', 'INSR', 'ITGA3', 'JAZ

### Question 5: What are the top significant SNPs for type 2 diabetes in the European population?

To answer this, we need to combine data from two different endpoints: `/v2/associations` and `/v2/studies`.

1.  First, we'll get a list of all associations for "type 2 diabetes", sorted by p-value.
2.  Then, for each of those associations, we will look up its study using the `/v2/studies/{accession_id}` endpoint.
3.  We will check the study's `discovery_ancestry` field to see if it matches "European".
4.  We will stop once we have found the top 10 associations that meet this ancestry criteria.

In [13]:
# --- Configuration ---
trait_of_interest = "type 2 diabetes mellitus"
population_of_interest = "European"
results_to_find = 10
top_associations_in_population = []

pd.options.display.float_format = '{:.10f}'.format

print(f"Searching for top {results_to_find} SNPs for '{trait_of_interest}' in '{population_of_interest}' ancestry studies...")

# --- Step 1: Get all associations for the trait using the new helper function ---
# The list is already sorted by p-value because we specified it in the helper function's API call.
all_trait_associations = get_all_associations_for_trait(trait_of_interest)

# --- Step 2: Loop through the sorted associations and check each study's ancestry ---
if all_trait_associations:
    for association in all_trait_associations:
        accession_id = association.get('accession_id')
        if not accession_id:
            continue

        # Fetch the study details to check its ancestry
        study_data = gwas_api_request(f"/v2/studies/{accession_id}")

        if study_data:
            discovery_ancestry = study_data.get('discovery_ancestry', [])
            if any(population_of_interest.lower() in s.lower() for s in discovery_ancestry):
                top_associations_in_population.append(association)
        
        # Stop once we have found enough results
        if len(top_associations_in_population) >= results_to_find:
            break

# --- Step 3: Display the final results ---
print(f"\n--- Search Complete ---")

# Set pandas display format for floats
pd.options.display.float_format = '{:.20f}'.format

if top_associations_in_population:
    print(f"Found the top {len(top_associations_in_population)} matching associations.")
    
    results_df = pd.DataFrame(top_associations_in_population)
    
    # Parse the risk allele field for cleaner display
    risk_allele_str = results_df['snp_risk_allele'].str[0]
    split_allele = risk_allele_str.str.split('-', n=1, expand=True)
    results_df['variant_rsID'] = split_allele[0]
    results_df['risk_allele_base'] = split_allele[1]
    
    # Display the final, cleaned-up DataFrame
    display(results_df[[
        'p_value',
        'variant_rsID',
        'risk_allele_base',
        'accession_id',
        'first_author'
    ]])
else:
    print("Could not find any matching associations.")

Searching for top 10 SNPs for 'type 2 diabetes mellitus' in 'European' ancestry studies...
--- Starting search for all associations related to 'type 2 diabetes mellitus' ---
Error: Received status code 500
URL: https://wwwdev.ebi.ac.uk/gwas/beta/rest/api/v2/associations?efo_trait=type+2+diabetes+mellitus&sort=p_value&direction=asc&size=200&page=0
Response: {"timestamp":"2025-07-25T15:20:01.990+00:00","status":500,"error":"Internal Server Error","path":"/gwas/beta/rest/api/v2/associations"}
No more data or an error occurred. Stopping.
--- Finished fetching. Found 0 total associations. ---


--- Search Complete ---
Could not find any matching associations.


### Question 6: Which variants are common between "type 2 diabetes" and "obesity"?

To find the variants associated with both traits, we'll follow these steps:

1.  Get all unique variants for "type 2 diabetes".
2.  Get all unique variants for "obesity".
3.  Compare the two sets of variants to find the ones they have in common.

In [14]:
import time

# --- Main script ---
# Define the two traits of interest
trait_1 = "type 2 diabetes mellitus"
trait_2 = "obesity"

# Get the set of variants for each trait
variants_for_trait_1 = get_all_variants_for_trait(trait_1)
variants_for_trait_2 = get_all_variants_for_trait(trait_2)

# Find the intersection of the two sets
common_variants = variants_for_trait_1.intersection(variants_for_trait_2)

# --- Display the final results ---
print("\n--- Search Complete ---")
if common_variants:
    print(f"Found {len(common_variants)} variants common to both '{trait_1}' and '{trait_2}':")
    # Display the list of common variants
    display(sorted(list(common_variants)))
else:
    print(f"No common variants found between '{trait_1}' and '{trait_2}'.")

--- Starting search for 'type 2 diabetes mellitus' ---
Found 7651 total associations across 39 pages.
Page 1/39 processed. Found 200 unique variants so far.
Page 2/39 processed. Found 400 unique variants so far.
Page 3/39 processed. Found 599 unique variants so far.
Page 4/39 processed. Found 798 unique variants so far.
Page 5/39 processed. Found 998 unique variants so far.
Page 6/39 processed. Found 1198 unique variants so far.
Page 7/39 processed. Found 1369 unique variants so far.
Page 8/39 processed. Found 1416 unique variants so far.
Page 9/39 processed. Found 1464 unique variants so far.
Page 10/39 processed. Found 1617 unique variants so far.
Page 11/39 processed. Found 1759 unique variants so far.
Page 12/39 processed. Found 1951 unique variants so far.
Page 13/39 processed. Found 2124 unique variants so far.
Page 14/39 processed. Found 2189 unique variants so far.
Page 15/39 processed. Found 2264 unique variants so far.
Page 16/39 processed. Found 2279 unique variants so far.


['rs11642015',
 'rs1229984',
 'rs1296328',
 'rs13028310',
 'rs13130484',
 'rs1412265',
 'rs1421085',
 'rs1556036',
 'rs17770336',
 'rs1955851',
 'rs2239647',
 'rs2306593',
 'rs2307111',
 'rs261966',
 'rs34811474',
 'rs34872471',
 'rs429358',
 'rs4776970',
 'rs4923464',
 'rs539515',
 'rs56094641',
 'rs6567160',
 'rs7132908',
 'rs7498798',
 'rs7903146',
 'rs8050136']

### Question 7: What are the most reported genes for a specific disease?

1. Get all associations for the trait
2. Extract and count all gene mentions
3. Display the top N most common genes

In [15]:
from collections import Counter

trait_of_interest = "type 2 diabetes mellitus"
results_to_find = 10

print(f"Searching for the most reported genes in associations for '{trait_of_interest}'...")

# --- Step 1: Get all associations for the trait ---
# This reuses the helper function from the previous question.
all_trait_associations = get_all_associations_for_trait(trait_of_interest)

# --- Step 2: Extract and count all gene mentions ---
all_gene_mentions = []
if all_trait_associations:
    for association in all_trait_associations:
        # Check if 'mapped_genes' exists and is not empty
        if 'mapped_genes' in association and association['mapped_genes']:
            # The field is a list, e.g., ['GENE1', 'GENE2,GENE3']
            for gene_string in association['mapped_genes']:
                # A single string can have multiple comma-separated genes
                genes = [gene.strip() for gene in gene_string.split(',') if gene.strip()]
                all_gene_mentions.extend(genes)

# --- Step 3: Display the top N most common genes ---
print(f"\n--- Analysis Complete ---")
if all_gene_mentions:
    # Use pandas Series and value_counts() for easy counting and display
    gene_counts = pd.Series(all_gene_mentions).value_counts()
    
    print(f"Found {len(gene_counts)} unique genes in total.")
    print(f"The top {results_to_find} most frequently reported genes for '{trait_of_interest}' are:")
    
    # Display the top genes
    display(gene_counts.head(results_to_find))
else:
    print(f"Could not find any gene associations for '{trait_of_interest}'.")


Searching for the most reported genes in associations for 'type 2 diabetes mellitus'...
--- Starting search for all associations related to 'type 2 diabetes mellitus' ---
Error: Received status code 500
URL: https://wwwdev.ebi.ac.uk/gwas/beta/rest/api/v2/associations?efo_trait=type+2+diabetes+mellitus&sort=p_value&direction=asc&size=200&page=0
Response: {"timestamp":"2025-07-25T15:22:18.033+00:00","status":500,"error":"Internal Server Error","path":"/gwas/beta/rest/api/v2/associations"}
No more data or an error occurred. Stopping.
--- Finished fetching. Found 0 total associations. ---


--- Analysis Complete ---
Could not find any gene associations for 'type 2 diabetes mellitus'.


### Question 8: What traits are associated with a particular SNP?

1. Paginate through all associations for the SNP
2. Extract the trait name from each association

In [16]:
import pandas as pd
import time


# Configuration
snp_of_interest = "rs1050316"
associated_traits = set() # Use a set to automatically store unique trait names

print(f"Searching for all traits associated with SNP '{snp_of_interest}'...")

# --- Step 1: Paginate through all associations for the SNP ---
current_page = 0
total_pages = 1 # Initialize to 1 to start the loop

while current_page < total_pages:
    endpoint = "/v2/associations"
    params = {
        "rs_id": snp_of_interest,
        "size": 200,
        "page": current_page
    }

    data = gwas_api_request(endpoint, params)
    
    if data and '_embedded' in data:
        associations_list = data['_embedded']['associations']
        
        # On the first request, find out the total number of pages
        if current_page == 0:
            total_pages = data['page']['totalPages']
            print(f"Found {data['page']['totalElements']} total associations across {total_pages} pages.")

        # --- Step 2: Extract the trait name from each association ---
        for association in associations_list:
            # The 'efo_traits' field is a list of dictionaries
            if 'efo_traits' in association and association['efo_traits']:
                for trait_info in association['efo_traits']:
                    if 'efo_trait' in trait_info:
                        associated_traits.add(trait_info['efo_trait'])
        
        print(f"Page {current_page + 1}/{total_pages} processed. Found {len(associated_traits)} unique traits so far.")
        current_page += 1
        time.sleep(0.1) # Be polite to the API
    else:
        # Stop if there's no more data or an error occurs
        print("No more data found or an error occurred. Stopping.")
        break

# --- Step 3: Display the final results ---
print(f"\n--- Search Complete ---")
if associated_traits:
    # Convert the set to a sorted list for clean display
    sorted_traits = sorted(list(associated_traits))
    
    print(f"Found a total of {len(sorted_traits)} unique traits associated with '{snp_of_interest}':")
    
    # Display the list of traits
    for trait in sorted_traits:
        print(f"- {trait}")
else:
    print(f"Could not find any traits associated with '{snp_of_interest}'.")

Searching for all traits associated with SNP 'rs1050316'...
Found 12 total associations across 1 pages.
Page 1/1 processed. Found 8 unique traits so far.

--- Search Complete ---
Found a total of 8 unique traits associated with 'rs1050316':
- Headache
- TPE interval measurement
- blood protein amount
- body height
- pain
- platelet count
- platelet crit
- serum alanine aminotransferase amount


### Question 9: Which studies on a trait have full summary statistics?

1. Paginate through all matching studies
2. Display the final results

In [17]:
trait_of_interest = "type 2 diabetes"
studies_with_sum_stats = [] # A list to store the resulting study objects

print(f"Searching for studies on '{trait_of_interest}' with full summary statistics...")

# --- Step 1: Paginate through all matching studies ---
current_page = 0
total_pages = 1 # Initialize to 1 to start the loop

while current_page < total_pages:
    endpoint = "/v2/studies"
    params = {
        "disease_trait": trait_of_interest,
        "full_pvalue_set": True, # Filter for studies with summary stats
        "size": 200,
        "page": current_page
    }

    data = gwas_api_request(endpoint, params)
    
    if data and '_embedded' in data:
        studies_list = data['_embedded']['studies']
        studies_with_sum_stats.extend(studies_list)
        
        # On the first request, find out the total number of pages
        if current_page == 0:
            total_pages = data['page']['totalPages']
            print(f"Found {data['page']['totalElements']} total matching studies across {total_pages} pages.")

        print(f"Page {current_page + 1}/{total_pages} processed. Collected {len(studies_with_sum_stats)} studies so far.")
        current_page += 1
        time.sleep(0.1) # Be polite to the API
    else:
        print("No more data found or an error occurred. Stopping.")
        break

# --- Step 2: Display the final results ---
print(f"\n--- Search Complete ---")
if studies_with_sum_stats:
    # Convert the list of studies to a DataFrame
    results_df = pd.DataFrame(studies_with_sum_stats)
    
    print(f"Found a total of {len(results_df)} studies for '{trait_of_interest}' with full summary statistics.")

    # Display relevant columns from the DataFrame
    display(results_df[[
        'accession_id',
        'pubmed_id',
        'initial_sample_size',
        'full_summary_stats' # This column contains the link to the stats
    ]])
else:
    print(f"Could not find any studies for '{trait_of_interest}' with full summary statistics.")

Searching for studies on 'type 2 diabetes' with full summary statistics...
Found 32 total matching studies across 1 pages.
Page 1/1 processed. Collected 32 studies so far.

--- Search Complete ---
Found a total of 32 studies for 'type 2 diabetes' with full summary statistics.


,accession_id,pubmed_id,initial_sample_size,full_summary_stats
0,GCST90528074,40210677,"23,541 Hispanic or Latino cases, 37,434 Hispan...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
1,GCST90444202,39379762,"51,256 African, African American, East Asian, ...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
2,GCST90302887,37377600,"22,670 British ancestry cases, 313,404 British...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
3,GCST90077724,34662886,"3,497 European ancestry cases, 328,257 Europea...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
4,GCST90296698,38182742,"21,507 Finnish ancestry female cases, 156,472 ...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
5,GCST90296697,38182742,"27,607 Finnish ancestry male cases, 118,687 Fi...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
6,GCST90161239,36329257,"3,844 Taiwanese ancestry cases, 59,333 Taiwane...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
7,GCST90006934,33188205,"9,978 European ancestry cases, 12,348 European...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
8,GCST90018926,34594039,"38,841 European ancestry cases, 451,248 Europe...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...
9,GCST90018706,34594039,"45,383 East Asian ancestry cases, 132,032 East...",http://ftp.ebi.ac.uk/pub/databases/gwas/summar...


### Question 10: What variants are in a gene `HBS1L` and what are their associated traits?

1. We'll use the `/v2/single-nucleotide-polymorphisms` endpoint to find all variants (SNPs) mapped to our gene of interest.

2. For each of those variants, we'll query the `/v2/associations` endpoint to find all the traits linked to it.

In [18]:
# --- Configuration ---
gene_of_interest = "HBS1L"

print(f"--- Step 1: Finding variants for gene '{gene_of_interest}' ---")

# --- Find all variants for the gene, handling pagination ---
gene_variants_rsids = []
current_page = 0
total_pages = 1
while current_page < total_pages:
    endpoint = "/v2/single-nucleotide-polymorphisms"
    params = {"gene": gene_of_interest, "size": 20, "page": current_page}
    data = gwas_api_request(endpoint, params)
    
    if data and '_embedded' in data:
        snp_list = data['_embedded']['snps']
        for snp in snp_list:
            if 'rs_id' in snp:
                gene_variants_rsids.append(snp['rs_id'])
        
        if current_page == 0:
            total_pages = data['page']['totalPages']
        print(f"Page {current_page + 1}/{total_pages} processed. Found {len(gene_variants_rsids)} variants so far.")
        current_page += 1
        time.sleep(0.1) # Be polite
    else:
        break

print(f"\nFound {len(gene_variants_rsids)} total variants for '{gene_of_interest}'.")

# --- Step 2: For the first few variants, find their associated traits ---
variant_trait_map = {}
variants_to_query = gene_variants_rsids[:len(gene_variants_rsids)]

print(f"\n--- Step 2: Checking for associated traits for the first {len(variants_to_query)} variants ---")

for rs_id in variants_to_query:
    traits_for_snp = set()
    endpoint = "/v2/associations"
    params = {"rs_id": rs_id, "size": 200}
    assoc_data = gwas_api_request(endpoint, params)
    
    if assoc_data and '_embedded' in assoc_data:
        for association in assoc_data['_embedded']['associations']:
            if 'efo_traits' in association and association['efo_traits']:
                for trait_info in association['efo_traits']:
                    if 'efo_trait' in trait_info:
                        traits_for_snp.add(trait_info['efo_trait'])
    
    variant_trait_map[rs_id] = sorted(list(traits_for_snp))
    print(f"Found {len(traits_for_snp)} traits for {rs_id}.")
    time.sleep(0.1) # Be polite

# --- Step 3: Display the results ---
print("\n--- Results ---")

# Convert the map to a list of dictionaries for DataFrame creation
display_data = [{'variant_rsID': key, 'associated_traits': value} for key, value in variant_trait_map.items()]
results_df = pd.DataFrame(display_data)

# Explode the list of traits into separate rows for better readability
results_df_exploded = results_df.explode('associated_traits').reset_index(drop=True)

if not results_df_exploded.empty:
    display(results_df_exploded)
else:
    print("Could not find any trait associations for the variants checked.")

--- Step 1: Finding variants for gene 'HBS1L' ---
Page 1/20754 processed. Found 20 variants so far.
Page 2/20754 processed. Found 40 variants so far.
Page 3/20754 processed. Found 60 variants so far.
Page 4/20754 processed. Found 80 variants so far.
Page 5/20754 processed. Found 100 variants so far.
Page 6/20754 processed. Found 120 variants so far.
Page 7/20754 processed. Found 140 variants so far.
Page 8/20754 processed. Found 160 variants so far.
Page 9/20754 processed. Found 180 variants so far.
Page 10/20754 processed. Found 200 variants so far.
Page 11/20754 processed. Found 220 variants so far.
Page 12/20754 processed. Found 240 variants so far.
Page 13/20754 processed. Found 260 variants so far.
Page 14/20754 processed. Found 280 variants so far.
Page 15/20754 processed. Found 300 variants so far.
Page 16/20754 processed. Found 320 variants so far.
Page 17/20754 processed. Found 340 variants so far.
Page 18/20754 processed. Found 360 variants so far.
Page 19/20754 processed. Fo

,variant_rsID,associated_traits
0,rs117741236,depressive disorder
1,rs1853229,depressive disorder
2,rs138801403,depressive disorder
3,rs56289435,depressive disorder
4,rs4786119,depressive disorder
...,...,...
4177,chr7:4521834,X-15503 measurement
4178,rs79868062,X-15503 measurement
4179,rs112844157,X-15503 measurement
4180,rs140858987,X-13728 measurement


### Question 12: Which variants are associated with an environmental factor on disease outcome? Eg: variants associated with smoking on lung cancer (GxE)


In [19]:
# Function to extract 'efo_trait' and join into a string
def extract_and_join_efo_traits(efo_list):
    if isinstance(efo_list, list):
        return ', '.join([d.get('efo_trait', '') for d in efo_list])
    return ''

In [20]:
# --- Configuration ---
trait_of_interest = "total blood protein measurement"
environmental_factor = "diet"
gxe_associations = []

print(f"===== Step 1: Finding GxE studies for '{trait_of_interest}' =====")

# --- Find all GxE studies for the trait, handling pagination ---
gxe_study_accessions = []
current_page = 0
total_pages = 1
while current_page < total_pages:
    endpoint = "/v2/studies"
    params = {
        "efo_trait": trait_of_interest,
        "gxe": True, # Filter for GxE studies
        "size": 200, 
        "page": current_page
    }
    data = gwas_api_request(endpoint, params)
    
    if data and '_embedded' in data:
        studies_list = data['_embedded']['studies']
        for study in studies_list:
            if 'accession_id' in study:
                gxe_study_accessions.append(study['accession_id'])
        
        if current_page == 0:
            total_pages = data['page']['totalPages']
        print(f"Page {current_page + 1}/{total_pages} processed. Found {len(gxe_study_accessions)} GxE studies.")
        current_page += 1
        time.sleep(0.1)
    else:
        break

print(f"\nFound {len(gxe_study_accessions)} GxE studies for '{trait_of_interest}'.")

# --- Step 2: Find associations with the environmental factor within those studies ---
print(f"\n===== Step 2: Searching for '{environmental_factor}' factor within these studies =====")
if gxe_study_accessions:
    for accession_id in gxe_study_accessions:
        # Get all associations for the current study
        study_associations = get_all_associations_for_accession_id(accession_id)
        
        for association in study_associations:
            efo_traits = association.get('efo_traits')
            if any([environmental_factor.lower() in t.get('efo_trait').lower() for t in efo_traits]):
                gxe_associations.append(association)
        print(f"Checked study {accession_id}, found {len(gxe_associations)} matching associations so far.")

# --- Step 3: Display the results ---
print(f"\n===== Step 3: Results =====")
if gxe_associations:
    print(f"Found {len(gxe_associations)} associations matching the GxE criteria.")
 
    results_df = pd.DataFrame(gxe_associations)
    
    # Apply the transformations
    results_df['reported_traits_formatted'] = results_df['reported_trait'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
    results_df['efo_traits_formatted'] = results_df['efo_traits'].apply(extract_and_join_efo_traits)
    
    pd.options.display.float_format = '{:.20f}'.format
    
    display(results_df[[
        'accession_id',
        'pubmed_id',
        'reported_traits_formatted',
        'efo_traits_formatted'
    ]])
    
    pd.reset_option('display.float_format')
else:
    print("Could not find any GxE associations matching the specified criteria.")

===== Step 1: Finding GxE studies for 'total blood protein measurement' =====
Page 1/1 processed. Found 3 GxE studies.

Found 3 GxE studies for 'total blood protein measurement'.

===== Step 2: Searching for 'diet' factor within these studies =====
--- Starting search for all associations related to 'GCST90161226' ---
Found 10 total associations across 1 pages.
Page 1/1 processed. Collected 10 associations so far.
--- Finished fetching. Found 10 total associations. ---

Checked study GCST90161226, found 10 matching associations so far.
--- Starting search for all associations related to 'GCST90161196' ---
Found 10 total associations across 1 pages.
Page 1/1 processed. Collected 10 associations so far.
--- Finished fetching. Found 10 total associations. ---

Checked study GCST90161196, found 20 matching associations so far.
--- Starting search for all associations related to 'GCST90026658' ---
Found 266 total associations across 2 pages.
Page 1/2 processed. Collected 200 associations so

,accession_id,pubmed_id,reported_traits_formatted,efo_traits_formatted
0,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
1,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
2,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
3,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
4,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
5,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
6,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
7,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
8,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"
9,GCST90161226,38990837,Total protein levels (adjusted for BMI) x vege...,"diet measurement, total blood protein measurement"


### Question 13: Which genomic region has the most variants associated with a disease? eg: the genomic region with most variants for type 2 diabetes

In [21]:
import pandas as pd
import time
from collections import Counter

# --- Configuration ---
trait_of_interest = "type 2 diabetes mellitus"
associations_to_check = 100
top_regions_to_display = 10

print(f"--- Step 1: Finding the top {associations_to_check} most significant associations for '{trait_of_interest}' ---")

data = get_all_associations_for_trait(trait_of_interest)
unique_rsids = set()

if data:
    for association in data:
        if 'snp_risk_allele' in association and association['snp_risk_allele']:
            rs_id = association['snp_risk_allele'][0].split('-')[0]
            if rs_id.startswith('rs'):
                unique_rsids.add(rs_id)

print(f"Found {len(unique_rsids)} unique variants from the top associations.")

# --- Step 2: For each unique variant, find its genomic region ---
region_list = []
if unique_rsids:
    print(f"\n--- Step 2: Looking up genomic regions for {len(unique_rsids)} variants ---")
    
    for i, rs_id in enumerate(list(unique_rsids)):
        snp_data = gwas_api_request(f"/v2/single-nucleotide-polymorphisms/{rs_id}")
        
        if snp_data and 'locations' in snp_data:
            for location in snp_data['locations']:
                if 'region' in location and 'name' in location['region']:
                    region_list.append(location['region']['name'])
        
        # Print progress update
        if (i + 1) % 500 == 0:
            print(f"Processed {i + 1}/{len(unique_rsids)} variants...")
        time.sleep(0.1) # Be polite to the API

# --- Step 3: Count and display the most common regions ---
print(f"\n--- Analysis Complete ---")
if region_list:
    # Use pandas Series and value_counts() for easy counting and display
    region_counts = pd.Series(region_list).value_counts()
    
    print(f"The top {top_regions_to_display} genomic regions with the most variants for '{trait_of_interest}' are:")
    display(region_counts.head(top_regions_to_display))
else:
    print("Could not retrieve region information for the variants.")

--- Step 1: Finding the top 100 most significant associations for 'type 2 diabetes mellitus' ---
--- Starting search for all associations related to 'type 2 diabetes mellitus' ---
Error: Received status code 500
URL: https://wwwdev.ebi.ac.uk/gwas/beta/rest/api/v2/associations?efo_trait=type+2+diabetes+mellitus&sort=p_value&direction=asc&size=200&page=0
Response: {"timestamp":"2025-07-25T15:55:35.620+00:00","status":500,"error":"Internal Server Error","path":"/gwas/beta/rest/api/v2/associations"}
No more data or an error occurred. Stopping.
--- Finished fetching. Found 0 total associations. ---

Found 0 unique variants from the top associations.

--- Analysis Complete ---
Could not retrieve region information for the variants.


### Question 14. Which SNP has the strongest effect size/OR for a disease? eg: The SNP with the strongest effect size/OR for type 2 diabetes

In [22]:
# --- Configuration ---
trait_of_interest = "type 2 diabetes mellitus"

print(f"Searching for the SNP with the strongest Odds Ratio for '{trait_of_interest}'...")

# --- Step 1: Query the API and sort by or_value ---
endpoint = "/v2/associations"
params = {
    "efo_trait": trait_of_interest,
    "sort": "or_value",      # Sort by Odds Ratio
    "direction": "desc",    # Sort descending to get the highest value first
    "size": 1               # We only need the top result
}

data = gwas_api_request(endpoint, params)

# --- Step 2: Display the result ---
if data and '_embedded' in data and data['_embedded']['associations']:
    # Extract the single top association
    top_association = data['_embedded']['associations'][0]
    
    print("\nFound the following association with the strongest effect size (Odds Ratio):")
    
    # For nice display, convert the single dictionary to a DataFrame
    results_df = pd.DataFrame([top_association])
    
    # Parse the risk allele field
    risk_allele_str = results_df['snp_risk_allele'].str[0]
    split_allele = risk_allele_str.str.split('-', n=1, expand=True)
    results_df['variant_rsID'] = split_allele[0]
    results_df['risk_allele_base'] = split_allele[1]

    # Set display format for floats
    pd.options.display.float_format = '{:.5f}'.format

    display(results_df[[
        'or_value',
        'p_value',
        'variant_rsID',
        'risk_frequency',
        'accession_id'
    ]])
    
    pd.reset_option('display.float_format')
else:
    print(f"\nCould not find an association with an Odds Ratio for '{trait_of_interest}'.")


Searching for the SNP with the strongest Odds Ratio for 'type 2 diabetes mellitus'...

Found the following association with the strongest effect size (Odds Ratio):


,or_value,p_value,variant_rsID,risk_frequency,accession_id
0,17.692,0.00000,rs77989445,0.1436,GCST006484


### Question 15. How many studies report a specific gene associated with a specific disease? eg: number of the studies reporting the KCNQ1 gene associated with type 2 diabetes

In [23]:
# --- Configuration ---
trait_of_interest = "type 2 diabetes mellitus"
gene_of_interest = "KCNQ1"
reporting_studies = set() # Use a set to store unique study accession_ids

print(f"Searching for the number of studies reporting '{gene_of_interest}' associated with '{trait_of_interest}'...")

# --- Step 1: Get all associations for the trait ---
# This reuses the helper function from previous questions.
# Make sure `get_all_associations_for_trait` is defined in your notebook.
all_trait_associations = get_all_associations_for_trait(trait_of_interest)

# --- Step 2: Filter associations by gene and collect unique study IDs ---
if all_trait_associations:
    for association in all_trait_associations:
        # Check if 'mapped_genes' exists and is not empty
        if 'mapped_genes' in association and association['mapped_genes']:
            # A single string can have multiple comma-separated genes, so we must check carefully
            found_gene = False
            for gene_string in association['mapped_genes']:
                if gene_of_interest in gene_string.split(','):
                    found_gene = True
                    break
            
            # If the gene was found, add the study's accession ID to our set
            if found_gene and 'accession_id' in association:
                reporting_studies.add(association['accession_id'])

# --- Step 3: Display the result ---
print(f"\n--- Search Complete ---")
if reporting_studies:
    final_count = len(reporting_studies)
    print(f"A total of {final_count} unique studies report the '{gene_of_interest}' gene in an association with '{trait_of_interest}'.")
    
    print("\nStudy Accession IDs:")
    # Display the list of unique study IDs
    for accession_id in sorted(list(reporting_studies)):
        print(f"- {accession_id}")
else:
    print(f"Could not find any studies reporting '{gene_of_interest}' for '{trait_of_interest}'.")

Searching for the number of studies reporting 'KCNQ1' associated with 'type 2 diabetes mellitus'...
--- Starting search for all associations related to 'type 2 diabetes mellitus' ---
Error: Received status code 500
URL: https://wwwdev.ebi.ac.uk/gwas/beta/rest/api/v2/associations?efo_trait=type+2+diabetes+mellitus&sort=p_value&direction=asc&size=200&page=0
Response: {"timestamp":"2025-07-25T15:55:37.932+00:00","status":500,"error":"Internal Server Error","path":"/gwas/beta/rest/api/v2/associations"}
No more data or an error occurred. Stopping.
--- Finished fetching. Found 0 total associations. ---


--- Search Complete ---
Could not find any studies reporting 'KCNQ1' for 'type 2 diabetes mellitus'.


### Question 16. What are the sample sizes used for the top 10 significant variants for disease, eg: find the top 10 significant variants associated with type 2 diabetes and find their sample size


In [24]:
# --- Configuration ---
trait_of_interest = "type 2 diabetes mellitus"
results_to_find = 10
final_results = []

print(f"--- Step 1: Finding the top {results_to_find} most significant associations for '{trait_of_interest}' ---")

# --- Get top 10 associations for the trait ---
endpoint = "/v2/associations"
params = {
    "efo_trait": trait_of_interest,
    "sort": "p_value",
    "direction": "asc",
    "size": results_to_find
}
data = gwas_api_request(endpoint, params)

if data and '_embedded' in data:
    top_associations = data['_embedded']['associations']
    print(f"Found {len(top_associations)} associations. Now fetching their study sample sizes...")

    # --- Step 2: For each association, find its study's sample size ---
    for association in top_associations:
        accession_id = association.get('accession_id')
        if not accession_id:
            continue

        # Make a second API call to get study details
        study_data = gwas_api_request(f"/v2/studies/{accession_id}")
        
        # Add the sample size to our association object
        if study_data and 'initial_sample_size' in study_data:
            association['initial_sample_size'] = study_data['initial_sample_size']
        else:
            association['initial_sample_size'] = "N/A"
        
        final_results.append(association)
        time.sleep(0.1) # Be polite to the API

# --- Step 3: Display the results ---
print(f"\n--- Search Complete ---")
if final_results:
    results_df = pd.DataFrame(final_results)
    
    # Parse the risk allele field
    risk_allele_str = results_df['snp_risk_allele'].str[0]
    split_allele = risk_allele_str.str.split('-', n=1, expand=True)
    results_df['variant_rsID'] = split_allele[0]
    
    # Set pandas display format for p-values
    pd.options.display.float_format = '{:.20f}'.format
    
    display(results_df[[
        'p_value',
        'variant_rsID',
        'initial_sample_size', # Display the sample size
        'accession_id'
    ]])
    
    pd.reset_option('display.float_format') # Reset display format
else:
    print(f"Could not retrieve top associations for '{trait_of_interest}'.")

--- Step 1: Finding the top 10 most significant associations for 'type 2 diabetes mellitus' ---
Found 10 associations. Now fetching their study sample sizes...

--- Search Complete ---


,p_value,variant_rsID,initial_sample_size,accession_id
0,0.00000000000000000000,rs7903146,"50,251 African American cases, 103,909 African...",GCST90492734
1,0.00000000000000000000,rs7903146,"148,726 European ancestry cases, 965,732 Europ...",GCST010555
2,0.00000000000000000000,rs35011184,"148,726 European ancestry cases, 24,646 Africa...",GCST010557
3,0.00000000000000000000,rs7903146,"51,256 African, African American, East Asian, ...",GCST90444202
4,0.00000000000000000000,rs2237897,"50,251 African American cases, 103,909 African...",GCST90492734
5,0.00000000000000000000,rs7903146,"251,740 European ancestry individuals, 139,705...",GCST90132183
6,0.00000000000000000000,rs7903146,"74,124 European ancestry cases, 824,006 Europe...",GCST009379
7,0.00000000000000000000,rs7766070,"50,251 African American cases, 103,909 African...",GCST90492734
8,0.00000000000000000000,rs10811661,"50,251 African American cases, 103,909 African...",GCST90492734
9,0.00000000000000000000,rs7903146,"61,714 European ancestry cases, 1,178 Pakistan...",GCST006867
